In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Загрузка данных:
# Данные загружаются из CSV файла.
# Удаляется колонка Unnamed: 0, если она присутствует.

diamonds = pd.read_csv('diamonds.csv')

if 'Unnamed: 0' in diamonds.columns:
    diamonds = diamonds.drop('Unnamed: 0', axis=1)

In [15]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [8]:
# Выделим признаки и целевую переменную
X = diamonds.drop('price', axis=1)
y = diamonds['price']

# Разделим признаки на числовые и категориальные
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Разделим данные на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создадим преобразователь для числовых и категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Преобразуем данные
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [9]:
param_grid = {'n_neighbors': range(1, 21, 5),
              'p': [1, 2]}
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

knn_best = grid_search.best_estimator_
y_pred_knn = knn_best.predict(X_test)
knn_mae = mean_absolute_error(y_test, y_pred_knn)

In [10]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
lin_reg_mae = mean_absolute_error(y_test, y_pred_lin)

In [11]:
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3)
sgd_reg.fit(X_train, y_train)
y_pred_sgd = sgd_reg.predict(X_test)
sgd_reg_mae = mean_absolute_error(y_test, y_pred_sgd)

In [14]:
print(f"KNN MAE: {knn_mae}")
print(f"Linear Regression MAE: {lin_reg_mae}")
print(f"SGD Regressor MAE: {sgd_reg_mae}")

best_model_name = min(
    [("KNN", knn_mae), ("Linear Regression", lin_reg_mae), ("SGD Regressor", sgd_reg_mae)], 
    key=lambda x: x[1]
)[0]

print(f"Best model based on MAE: {best_model_name}")

KNN MAE: 382.4982542330985
Linear Regression MAE: 737.1513665933436
SGD Regressor MAE: 731.9208437566717
Best model based on MAE: KNN


Вывод:

На основе метрики MAE (Mean Absolute Error) для различных моделей, можно сделать следующий вывод: исходя из значений MAE, наилучшей моделью является KNN, так как она имеет наименьшее значение MAE (382.4982542330985), что указывает на наименьшую среднюю абсолютную ошибку среди всех рассмотренных моделей. Таким образом, для задачи регрессии с данными бриллиантов модель KNN является предпочтительной по сравнению с Linear Regression и SGD Regressor.